In [29]:
import sys
import os
import pandas as pd

# Add the 'src' directory to the Python path
src_path = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_path)

# Now you can import your function
from data_collection import fetch_data_from_fred

# Define your FRED API key
api_key = 'e916710d165717e6348556cdce8111f3'

# Define the series IDs for the indicators you want to collect
series_ids = {
    'Unemployment_Rate': 'UNRATE',
    'GDP_Growth': 'A191RL1Q225SBEA',
    'CPI': 'CPIAUCSL',
    'Interest_Rate': 'FEDFUNDS',
    'Money_Supply': 'M2SL',
    'PPI': 'PPIACO',
    'Consumer_Confidence': 'UMCSENT'
}

# Define output directory for files
output_dir = 'C:/Users/ghkjs/OneDrive/바탕 화면/EconomicIndicatorPrediction/data/processed/FillingMissingValue'

# Show plot for each data series and save as CSV and PNG files
for name, series_id in series_ids.items():
    # Fetch the data
    data = fetch_data_from_fred(series_id, api_key)

    # Display the dataset to check for missing values
    print("Before interpolation:")
    print(data.isnull().sum())
    
    # Apply linear interpolation for missing values
    data_interpolated = data.interpolate(method='linear')
    
    # Display the dataset again to confirm missing values are filled
    print("\nAfter interpolation:")
    print(data_interpolated.isnull().sum())

    # Set the index name to 'Date'
    data_interpolated.index.name = 'Date'
    
    # Save the file as csv
    data_interpolated_csv_file_path = os.path.join(output_dir, f'{name}_linear_interpolation.csv')
    data_interpolated.to_csv(data_interpolated_csv_file_path, index=True)
    print(f'Data saved to {data_interpolated_csv_file_path}')
    print(data_interpolated.head())
    print(data_interpolated.tail())

print(f"Data columns for {name}: {data.columns}")


Before interpolation:
UNRATE    0
dtype: int64

After interpolation:
UNRATE    0
dtype: int64
Data saved to C:/Users/ghkjs/OneDrive/바탕 화면/EconomicIndicatorPrediction/data/processed/FillingMissingValue\Unemployment_Rate_linear_interpolation.csv
            UNRATE
Date              
1948-01-01     3.4
1948-02-01     3.8
1948-03-01     4.0
1948-04-01     3.9
1948-05-01     3.5
            UNRATE
Date              
2024-06-01     4.1
2024-07-01     4.3
2024-08-01     4.2
2024-09-01     4.1
2024-10-01     4.1
Before interpolation:
A191RL1Q225SBEA    0
dtype: int64

After interpolation:
A191RL1Q225SBEA    0
dtype: int64
Data saved to C:/Users/ghkjs/OneDrive/바탕 화면/EconomicIndicatorPrediction/data/processed/FillingMissingValue\GDP_Growth_linear_interpolation.csv
            A191RL1Q225SBEA
Date                       
1947-04-01             -1.0
1947-07-01             -0.8
1947-10-01              6.4
1948-01-01              6.2
1948-04-01              6.8
            A191RL1Q225SBEA
Date       